<a href="https://colab.research.google.com/github/s110/CollabNotebooks/blob/main/Maestria/MachineLearning/Maestr%C3%ADa_Hard_SVM_2024_21ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # Practice  Hard SVM.
 ----
  
  University : UTEC \\
  Course       : Machine Learning \\
  Professor    : Cristian López Del Alamo \\
  Topic        : Hard SVM \\
  Termina      : 12:45
   

 ----

Write the names and surnames of the members and the percentage of participation of each one in the development of the practice:
 - Integrante 1: (%)
 - Integrante 2: (%)
 - Integrante 3: (%)
 - Integrante 4: (%)


 ----


  



In [1]:
# Loading libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cvxopt
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import metrics

# Lagrange Multipliers

$\frac{\partial f(x)}{ \partial x} = λ \frac{\partial g(  x)}{ \partial x}$

----
Find the values of  $λ_i$ for each training elements $X_i$.

The  ***GetLambda*** function returns a vector that we will call  lambda, such that   $lambda[i]$ will be  $0$, if the element  $X[i]$ does not intersect with any of the lines   $XW^t + b >=1$ o $XW^t + b >=0$

Note: X is a matrix, so $X_i$ will be a  $K$-dimensional vector that represent the  i-th  object or  $k$-dimensional point, and  $X_{ij}$ is  the  j-th  element of the  i-th objet.

----


- **Note: The code for finding the lambda values is provided to you.**

In [2]:
import numpy as np
from cvxopt import matrix, solvers

def GetLambda(X, y):
    n, m = X.shape
    y = y.astype(float)
    K = np.dot(X, X.T) * np.dot(y, y.T)  # Kernel
    P = matrix(K)
    q = matrix(-np.ones(n))
    G = matrix(-np.eye(n))
    h = matrix(np.zeros(n))
    A = matrix(y.reshape(1, -1))
    b = matrix(np.zeros(1))
    sol = solvers.qp(P, q, G, h, A, b)
    alpha = np.array(sol['x'])
    return alpha

#Ejemplo para utilizar esta función
#lamda = GetLambda(X,Y)
#sv = lamda > 1e-5
#print(sv)

## 2 Calculation of the Weights W
$W_j = \sum_{i=0}^n \lambda_iy_ix_{ij}$  

----
Where: $λ_i$ represent  $i-th$ lagrange multiplier, $W_j$ is the $j-th$ weight,   $x_{ij}$ denotes the value of feacture $(j)$ for the $(i)-th$ training objetc, and $y_i$ is the expected output (1 or -1) for the $i-th$ object.

$W_j = \sum_{i=0}^n \lambda_iy_ix_{ij}$  
Note that the summation only includes elements for which the Lagrange

----

multiplier $lamnda_i$ is nonzero.



In [3]:
import numpy as np

def Get_W(X, Y, lambda_list):
  """
  Calcula el vector de pesos W para un SVM lineal.

  Args:
    X (np.ndarray): Matriz de datos de entrenamiento, donde cada fila es una
                    muestra y cada columna es una característica.
                    Dimensiones: (n_samples, n_features).
    Y (np.ndarray): Vector de etiquetas de clase (1 o -1) para cada muestra.
                    Dimensiones: (n_samples,) o (n_samples, 1).
    lambda_list (np.ndarray): Vector de multiplicadores de Lagrange.
                              Dimensiones: (n_samples,) o (n_samples, 1).

  Returns:
    np.ndarray: El vector de pesos W. Dimensiones: (n_features,).
  """
  # Asegurarse de que lambda_list y Y sean arrays 1D para cálculos consistentes.
  # La salida de GetLambda (sol['x']) es una matriz cvxopt, luego convertida a np.array,
  # que probablemente tiene forma (n_samples, 1). Usamos flatten().
  # Y también podría ser (n_samples,) o (n_samples, 1).
  lambda_flat = lambda_list.flatten()
  Y_flat = Y.flatten()

  # n_samples, n_features = X.shape # No es estrictamente necesario aquí

  # La fórmula es W_j = sum_i (lambda_i * y_i * x_ij)
  # Esto se puede vectorizar como W = X^T @ (lambda * y)

  # 1. Calcular el término (lambda_i * y_i) para cada muestra.
  # Este término pondera la contribución de cada muestra.
  # Si lambda_i es cercano a 0 (vector no de soporte), su contribución es mínima.
  weighted_terms = lambda_flat * Y_flat

  # 2. Calcular W.
  # W_j = sum_i (weighted_terms_i * X_ij)
  # Esto es equivalente a la transpuesta de X multiplicada por los términos ponderados.
  # X.T tiene dimensiones (n_features, n_samples)
  # weighted_terms tiene dimensiones (n_samples,)
  # El resultado W tendrá dimensiones (n_features,)
  W = np.dot(X.T, weighted_terms)

  return W

## Finding the Bias (b)

$XW^t + b = 0$

$b = - \frac{1}{n}∑_{i=0}^n X_iW^t$

Where $X_i$ is a $k$-dimensional vector representing the $i$-th object, and $k$ is the number of features of the object.



In [4]:
def Get_b(X, W):
  """
  Calcula el término de sesgo b para un SVM lineal usando la fórmula
  b = - (1/n) * sum(X_i @ W).

  Args:
    X (np.ndarray): Matriz de datos de entrenamiento, donde cada fila es una
                    muestra y cada columna es una característica.
                    Dimensiones: (n_samples, n_features).
    W (np.ndarray): Vector de pesos W. Asumimos que es un array 1D.
                    Dimensiones: (n_features,).

  Returns:
    float: El término de sesgo b.
  """
  # n_samples es el número de filas en X
  n_samples = X.shape[0]

  # W es un vector 1D (n_features,).
  # X es una matriz (n_samples, n_features).
  # np.dot(X, W) calcula el producto punto de cada fila de X con W.
  # El resultado es un array 1D de longitud n_samples, donde cada
  # elemento es X_i @ W^T (o X_i · W).
  # Ejemplo:
  # X = [[x11, x12], [x21, x22]]
  # W = [w1, w2]
  # np.dot(X, W) = [x11*w1 + x12*w2, x21*w1 + x22*w2]
  xw_products = np.dot(X, W)

  # Sumamos todos estos productos punto
  sum_xw_products = np.sum(xw_products)

  # Calculamos el promedio de los productos punto
  mean_xw_products = sum_xw_products / n_samples

  # b es el negativo de este promedio
  b = -mean_xw_products

  return b

## Testing Stage

----
For this stage, one should only calculate :

- $f(X_j) = X_jW^t + b$

But since we have already calculated the values of the parameters $W$ and  $b$, then by substituting we have :

- $f(X_j) = \sum_{i=0}^n \lambda_iy_i<X_{i},X_{j}> + b$

Donde: $X_i$ is the i-th  training vector and  $X_j$   is the new vector that passes through the model for predicting the class (1 or -1)

Finally, to determine which class the new vector $X_j$   belongs to, it is sufficient to check the sign of f(X_j).

  - **If $f(X_j) >=0$ then $Y_j$ = 1 else $Y_j = -1$**
  -----

In [5]:
def Test(X_new, W, b):
  """
  Predice las etiquetas de clase para nuevas muestras utilizando un modelo SVM lineal
  previamente entrenado (con W y b conocidos).

  Args:
    X_new (np.ndarray): Matriz de nuevas muestras a clasificar.
                        Cada fila es una muestra, cada columna es una característica.
                        Dimensiones: (n_new_samples, n_features).
    W (np.ndarray): Vector de pesos del SVM.
                    Asumimos que es un array 1D.
                    Dimensiones: (n_features,).
    b (float): Término de sesgo del SVM.

  Returns:
    np.ndarray: Un vector de etiquetas de clase predichas (1 o -1)
                para cada muestra en X_new.
                Dimensiones: (n_new_samples,).
  """
  # Calcular la función de decisión para cada muestra en X_new:
  # f(X_j) = X_j @ W + b
  # np.dot(X_new, W) calcula el producto punto de cada fila de X_new con W.
  # Si X_new es (m, k) y W es (k,), el resultado es (m,).
  decision_scores = np.dot(X_new, W) + b

  # Aplicar la regla de clasificación:
  # Si f(X_j) >= 0, entonces Y_j = 1
  # Si f(X_j) < 0, entonces Y_j = -1
  # La función np.where es ideal para esto.
  predictions = np.where(decision_scores >= 0, 1, -1)

  return predictions

Database for Testing:
[Download](https://docs.google.com/spreadsheets/d/15-E3kiLJ6bCyXuJvSmxYAp2QYMkPX2QlQ597fAsPYy8/edit#gid=0).

----
Download the database to your disk and use files.upload() to load it onto the drive. The code is provided.
----


- Split the dataset into 70% for training and 30% for testing.
- Add a value of 1 for the first class and -1 for the second class.
- In the testing stage, find the number of elements correctly classified and the number of elements incorrectly classified for each class

- Create a [confusion matrix](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html) which will show us the efficiency of the method.

- Do not forget to normalize the data.

- Plot the lines that separate both classes.

----


In [14]:
# code for loading  the Dataset
import pandas as pd
import numpy as np

url='https://github.com/s110/CollabNotebooks/raw/refs/heads/main/Maestria/MachineLearning/SMV/DataSet_Iris_2_Clases.csv'
data = pd.read_csv(url)

X = data[["sepal.length","sepal.width","petal.length","petal.width"]]
Y = data[["variety"]]
print(X)

# Splitting the dataset into training and testing

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y , random_state=104,  test_size=0.30,    shuffle=True)




UnicodeDecodeError: 'utf-8' codec can't decode bytes in position 10-11: invalid continuation byte

In [ ]:
# Obtaining the values of the parameters W and b, to predict the classes to which the values of X_test belong

W =  Get_W(X_test,Y_test,lambda_list)
b =  Get_b(X_test,W)

# Convert all values greater than 0 to 1, and those less than 0 to -1
Y_pred  = np.sign(Test(X_test,W,b))

# We create a confution matrix
confusion_matrix = metrics.confusion_matrix(Y_test, Y_pred)
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [1, -1])
cm_display.plot()
plt.show()


----

- Subir el link de su colab a canvas
- Disfruten aprendiendo. La única forma de aprender es haciendo.
- Buena Suerte.
----